In [ ]:
! pip install -q -U google-generativeai

In [ ]:

import os
from dotenv import load_dotenv  # helps to load env files

import google.generativeai as genai
# Load all env files
load_dotenv()

GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
GOOGLE_API_KEY

In [ ]:
list(genai.list_models())

### Perform Embedding on single string

In [ ]:
from typing import Dict

result :Dict = genai.embed_content(
    model="models/text-embedding-004",
    # This content will goona convert into embedding 
    content="What is the meaning of life?",
    task_type="retrieval_document",
    title="Embedding of single string"
)

result['embedding']

In [ ]:
len(result['embedding'])

### Perform Embedding on list of string

In [ ]:
from typing import Dict
result:Dict = genai.embed_content(
    model="models/text-embedding-004",
    content=[
        'what is meaning of life?',
        "How does the brain work?",
        "Explain difference between CPU and GPU?"
    ],
    task_type="retrieval_document",
    title="Embedding of list of string"

)


len(result['embedding'])


In [ ]:
for embedded_result in result['embedding']:
    print(str(embedded_result)[:50], " ..... Trimmed ...... ", len(embedded_result))

### Building Vector Store & Reterival using Chroma DB and LangChain

In [ ]:
! pip install -Uq langchain-chroma

In [ ]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Roses are one of the most popular flowers, symbolizing love and passion. They come in various colors, each carrying different meanings. Proper care includes regular watering, pruning, and exposure to sunlight.",
        metadata={"source": "flower-garden-docs"}
    ),
    Document(
        page_content="Tulips are vibrant spring flowers that bloom in a variety of colors. They thrive in well-drained soil and require full sun to partial shade. Tulips are often associated with elegance and new beginnings.",
        metadata={"source": "flower-garden-docs"}
    ),
    Document(
        page_content="Lotuses are aquatic flowers that symbolize purity and enlightenment. They grow in ponds and still waters, requiring warm temperatures and full sunlight. Their leaves repel water, keeping them dry even in wet conditions.",
        metadata={"source": "aquatic-flowers-docs"}
    ),
    Document(
        page_content="Sunflowers are known for their large, bright yellow blooms that follow the sun. They require direct sunlight and well-drained soil to thrive. Sunflowers symbolize happiness, warmth, and positivity.",
        metadata={"source": "field-flowers-docs"}
    ),
    Document(
        page_content="Orchids are exotic flowers admired for their intricate beauty and variety. They require a humid environment, indirect sunlight, and well-ventilated spaces. Orchids symbolize love, strength, and refinement.",
        metadata={"source": "flower-garden-docs"}
    ),
]


In [ ]:
! pip install -Uq langchain-google-genai

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001" , 
    google_api_key=os.getenv('GOOGLE_API_KEY')
)

embeddings , os.getenv('GOOGLE_API_KEY')

In [ ]:
result = embeddings.embed_query("Rose Color")

len(result)

In [ ]:
# Here it use google gemini embedding model

from langchain_chroma import Chroma

vector_store = Chroma.from_documents(documents , embedding=embeddings)

vector_store

In [ ]:
list(dir(vector_store))

In [ ]:
vector_store.similarity_search("Tulip")

In [ ]:
await vector_store.asimilarity_search("tulip")

In [ ]:
vector_store.similarity_search_with_score("tulip")

In [ ]:
# Similarity Serach By Vector
new_embd = embeddings.embed_query("Tulip")

# we can also search by vector 
vector_store.similarity_search_by_vector(new_embd)

### Retrivers

In [ ]:
# when i run query at that time its object will create and use and then automatically remove that object from memory
from langchain_core.runnables import RunnableLambda
from langchain_core.documents import Document

# Return top most whose cosine angle are closer to each other 
reteriver = RunnableLambda(vector_store.similarity_search).bind(k=1)

reteriver.batch(['tulip']) 



In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key= GOOGLE_API_KEY)

llm

In [ ]:
# llm give huge response but our rag return minimum response , which is exist in my context window 
llm_response = llm.invoke("Tell me variety of tulip ? and where i can get tulip and also tell me tulip flower process?")

print(llm_response.content)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# This runnablePassThrough help to directly pass our question to llms and
# which means we are make dynamically prompt template so  it means when question resceive it will dynamically add into query 
from langchain_core.runnables import RunnablePassthrough



message = """
    You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. 
    If you don't know the answer, just say that you don't know.
    If possible can u please give me answer in bullet points . Make sure your answer is relevent to the question ans it is answered from the contetx only.
    Question: {question} 
    Context: {context} 
"""

prompt = ChatPromptTemplate.from_messages([("human" , message)])
prompt

### Prepare RAG Chain or RAG Pipeline

In [ ]:
# import output parser 
# with the help of Stroutputparser we can able to get final respone in string format 
from langchain_core.output_parsers import StrOutputParser

In [ ]:
rag_chain = {"context":reteriver , "question":RunnablePassthrough()} | prompt | llm | StrOutputParser()

rag_chain
    

In [ ]:
# question = "Tell me variety of tulip ? and where i can get tulip and also tell me tulip flower process?"

# question="Tell me about Aakash Prajapati"
question="Tell me about shark"
response = rag_chain.invoke(question)

# llm give huge response but our rag return minimum response , which is exist in my context window 
print(response)